LDA
- 로우별로 LDA(책마다) > 토픽을 5개~10개 뽑고 토픽별로 군집화 
- LDA는 책별로, 군집화는 전체로 

토큰화 후 전처리
- 길이가 3이하인 단어 제거? 한국어에서도 그렇게 처리하는게 맞을지 (보류, 진행안함)

목적
- 책들의 카테고리를 새로 분류하기 위해서 
- 결과는 A책 토픽 5개, B책 토픽 5개, C책 토픽 5개 =군집=> 한 군집에 A책과 C책이 묶일 수 있다 등 

to do
- 토큰화 제대로 명사만 뽑거나, 지금은 토큰화가 제대로 안되어있어 불용어도 제대로 처리되지 않음
- 표제어 추출은 왜 하는지?
- 각 토픽에 뽑히는 단어들의 기준 , 단순하게 10000개 뽑은 거 같은데 빈도수별 상위 100개 뽑기 등으로 수정해야함
    - 빈도 상위 1000개 단어
    
- DTM(CountVectorizer) vs TF-IDF(TfidfVectorizer)

어간추출, 표제어추출은 나중에 성능 향상때


앞에는 countvec
뒤에는 tf-idf

In [ ]:
#! pip install nltk

In [ ]:
#!pip install scikit-learn

In [ ]:
# import nltk
# nltk.download('punkt')

In [ ]:
# nltk.download('wordnet')

In [1]:
import pandas as pd
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
import pickle
with open("./lda_df.pickle","rb") as a:
    lda_df = pickle.load(a) 

In [ ]:
lda_df.info()

In [3]:
lda_df2 = lda_df[:1000]

In [4]:
text = lda_df2['book_introduce']

### 토큰화

In [ ]:
from konlpy.tag import * 

In [ ]:
from tqdm import tqdm
okt = Okt()

tokens = []
for i in tqdm(range(len(text))):
    token = okt.nouns(text[i])
    tokens.append(token)
    

In [ ]:
tokens

### 불용어 제거

In [ ]:
stopwords = ['와','에','으로','도','를','과','는','의','가','이','은','이','있','하','것','들','그','되','수','이','보','않','없','나','사람','주','아니','등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나','받','못하','일','그런','또','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓']

In [ ]:
drop_stopwords = []
for j in range(len(text)):
    stop = [word for word in tokens[j] if word not in stopwords]
    drop_stopwords.append(stop)


In [ ]:
drop_stopwords

길이가 한글자인 단어 제거 -> 보류

In [ ]:
# drop_one = []
# for k in range(len(text)):
#     one = [word for word in drop_stopwords[k] if len(word) > 1]
#     drop_one.append(one)

In [ ]:
# drop_one

In [ ]:
# 로우(책)마다 토픽을 뽑아보기 - 샘플로 첫번째 책만 
drop_stopwords[0]

역토큰화

In [ ]:
# 역토큰화 
detokenized_doc = []
t = ' '.join(drop_stopwords[0])
detokenized_doc.append(t)

detokenized_doc

In [ ]:
# 역토큰화 
# detokenized_doc = []
# for i in range(len(text)):
#     t = ' '.join(text[i])
#     detokenized_doc.append(t)

# text= detokenized_doc

워드임베딩

In [ ]:
# TF-IDF
# 문서 내 특정 단어의 중요도
# 문서 단어 행렬(DTM) 만들고 TF-IDF
# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words=stopwords, max_features= 1000).fit(detokenized_doc)
#X = vectorizer.fit_transform(detokenized_doc)
X = vectorizer.transform(detokenized_doc)

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)
# 가중치
print(X.toarray())
print(vectorizer.vocabulary_)

In [ ]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [ ]:
lda_top = lda_model.fit_transform(X)

In [ ]:
# 단어의 확률분포
print(lda_model.components_)
print(lda_model.components_.shape) 

In [ ]:
terms = list(vectorizer.vocabulary_.keys())

# 토픽 확인 함수
def get_topics(components, feature_names, n=1000):
    features_li = []
    for idx, topic in enumerate(components):
#         print("Topic %d:" % (idx + 1))
#         print(", ".join([feature_names[i] for i in topic.argsort()[:-n - 1:-1]]))
        
        print("Topic %d:" % (idx+1))
        print([(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        
        features = []
        for i in topic.argsort()[:-n - 1:-1]:
            feature = feature_names[i]
            features.append(feature)
            
        features_li.append(features)
    return features_li


In [ ]:
features_li = get_topics(lda_model.components_,terms)

In [ ]:
print(features_li)
# 10개의 토픽에 대한 확률이 높은순의 단어들 

In [5]:
# 반복문
from konlpy.tag import * 
from tqdm import tqdm


# 토큰화
okt = Okt()

tokens = []
for i in tqdm(range(len(text))):
    token = okt.nouns(text[i])
    tokens.append(token)

# 불용어 
stopwords = ['와','에','으로','도','를','과','는','의','가','이','은','이','있','하','것','들','그','되','수','이','보','않','없','나','사람','주','아니','등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나','받','못하','일','그런','또','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓']

drop_stopwords = []
for j in range(len(text)):
    stop = [word for word in tokens[j] if word not in stopwords]
    drop_stopwords.append(stop)


# 토픽 확인 함수
def get_topics(components, feature_names, n=1):
    features_li = []
    for idx, topic in enumerate(components):
        
        features = []
        for i in topic.argsort()[:-n - 1:-1]:
            feature = feature_names[i]
            features.append(feature)
            
        features_li.append(features)
    return features_li
        

    
topics_li = []
for idx in range(len(drop_stopwords)):
    
    detokenized_doc = []
    t = ' '.join(drop_stopwords[idx])
    detokenized_doc.append(t)

    # 상위 1,000개의 단어를 보존 
    vectorizer = TfidfVectorizer(stop_words=stopwords, max_features=1000)
    X = vectorizer.fit_transform(detokenized_doc)

    lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
    lda_top = lda_model.fit_transform(X)

    terms = list(vectorizer.vocabulary_.keys())

    topics = get_topics(lda_model.components_,terms)
    flattened_topics = [word for sublist in topics for word in sublist]
    topics_li.append(flattened_topics)
      
print(topics_li)

100%|███████████████████████████████████████| 1000/1000 [03:52<00:00,  4.31it/s]


[['바로', '경제', '분량', '연쇄', '무려', '자동화', '달성', '초판', '초반', '육성'], ['아이러니', '영위', '공감', '전하', '얼마나', '제대로', '전하', '욕망', '유튜버', '현실'], ['이번', '구성', '구성', '합격', '주제', '제자', '정리', '세계', '어려움', '철학'], ['일자리', '필요', '논문', '난리', '폭발', '찬사', '비롯', '일자리', '매체', '마구'], ['제대로', '수출입', '수출입', '성과', '연방', '각종', '변동', '미국', '모든', '경제'], ['추가', '부자', '모습', '그날', '통해', '내용', '메시지', '과일', '일명', '대한'], ['대표', '장악', '신용', '통찰', '일생', '대체', '소개', '금융', '이해', '좇는'], ['크게', '이슈', '본질', '삼성', '자의', '해당', '삽시간', '도래', '삼성', '지식'], ['과연', '민영화', '확대', '어려움', '경제학', '베스트셀러', '이자', '제조업', '제조업', '경제'], ['댓글', '가장', '곳곳', '불안', '진정', '사이', '보고', '목적', '자극', '세대'], ['세션', '자산', '파격', '아버지', '아빠', '사람과', '사고방식', '세계', '사람과', '기존'], ['개인', '저축은행', '정부', '결론', '방송', '위협', '뉴스', '국부론', '의미', '정리'], ['업무', '소개', '신설', '주택', '가장', '라면', '자로', '청약', '업데이트', '자산'], ['무려', '장주', '타이밍', '수익', '참가', '평균', '수익', '수익률', '단계', '퍼센트'], ['부의', '유튜버', '칼국수', '직원', '중요성', '이벤트', '당첨', '강조', '이벤트', '시기'], ['구매', '도전', '심리', '자동화', '처음

군집

In [6]:
topics_li

[['바로', '경제', '분량', '연쇄', '무려', '자동화', '달성', '초판', '초반', '육성'],
 ['아이러니', '영위', '공감', '전하', '얼마나', '제대로', '전하', '욕망', '유튜버', '현실'],
 ['이번', '구성', '구성', '합격', '주제', '제자', '정리', '세계', '어려움', '철학'],
 ['일자리', '필요', '논문', '난리', '폭발', '찬사', '비롯', '일자리', '매체', '마구'],
 ['제대로', '수출입', '수출입', '성과', '연방', '각종', '변동', '미국', '모든', '경제'],
 ['추가', '부자', '모습', '그날', '통해', '내용', '메시지', '과일', '일명', '대한'],
 ['대표', '장악', '신용', '통찰', '일생', '대체', '소개', '금융', '이해', '좇는'],
 ['크게', '이슈', '본질', '삼성', '자의', '해당', '삽시간', '도래', '삼성', '지식'],
 ['과연', '민영화', '확대', '어려움', '경제학', '베스트셀러', '이자', '제조업', '제조업', '경제'],
 ['댓글', '가장', '곳곳', '불안', '진정', '사이', '보고', '목적', '자극', '세대'],
 ['세션', '자산', '파격', '아버지', '아빠', '사람과', '사고방식', '세계', '사람과', '기존'],
 ['개인', '저축은행', '정부', '결론', '방송', '위협', '뉴스', '국부론', '의미', '정리'],
 ['업무', '소개', '신설', '주택', '가장', '라면', '자로', '청약', '업데이트', '자산'],
 ['무려', '장주', '타이밍', '수익', '참가', '평균', '수익', '수익률', '단계', '퍼센트'],
 ['부의', '유튜버', '칼국수', '직원', '중요성', '이벤트', '당첨', '강조', '이벤트', '시기'],
 ['구매', '도전', '

In [7]:
import pandas as pd

df = pd.DataFrame(columns=['nouns'])
df

,nouns


In [8]:
df['nouns'] = topics_li

In [9]:
df

,nouns
0,"[바로, 경제, 분량, 연쇄, 무려, 자동화, 달성, 초판, 초반, 육성]"
1,"[아이러니, 영위, 공감, 전하, 얼마나, 제대로, 전하, 욕망, 유튜버, 현실]"
2,"[이번, 구성, 구성, 합격, 주제, 제자, 정리, 세계, 어려움, 철학]"
3,"[일자리, 필요, 논문, 난리, 폭발, 찬사, 비롯, 일자리, 매체, 마구]"
4,"[제대로, 수출입, 수출입, 성과, 연방, 각종, 변동, 미국, 모든, 경제]"
...,...
995,"[중상, 등장, 경제학자, 이론, 망라, 경제학, 그림, 시작, 최근, 경제]"
996,"[듀레이션, 실제, 분석, 수익률, 모든, 모든, 실제, 사용, 처음, 초점]"
997,"[대한, 인기, 전체, 안목, 전문가, 유망, 투자자, 세분, 무엇, 특성]"
998,"[사례, 실전, 사건, 대한, 등기, 우선, 내용증명, 발급, 특수, 소송]"


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df=1, max_df=0.95)

tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

In [13]:
new_df = pd.DataFrame(vector, columns=tfidf_vectorizer.get_feature_names_out())
new_df.head()

,가가,가감,가게,가격,가까이,가나다순,가난,가능,가능성,가동,...,휴지,흐름,흔적,흡수,흥망,흥미,흥미진진,희망,히데,힌트
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=3).fit(vector)
df['cluster'] = kmeans.labels_

/usr/local/Caskroom/miniconda/base/envs/my_env2/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [19]:
df

,nouns,cluster
0,"[바로, 경제, 분량, 연쇄, 무려, 자동화, 달성, 초판, 초반, 육성]",2
1,"[아이러니, 영위, 공감, 전하, 얼마나, 제대로, 전하, 욕망, 유튜버, 현실]",2
2,"[이번, 구성, 구성, 합격, 주제, 제자, 정리, 세계, 어려움, 철학]",2
3,"[일자리, 필요, 논문, 난리, 폭발, 찬사, 비롯, 일자리, 매체, 마구]",2
4,"[제대로, 수출입, 수출입, 성과, 연방, 각종, 변동, 미국, 모든, 경제]",2
...,...,...
995,"[중상, 등장, 경제학자, 이론, 망라, 경제학, 그림, 시작, 최근, 경제]",2
996,"[듀레이션, 실제, 분석, 수익률, 모든, 모든, 실제, 사용, 처음, 초점]",2
997,"[대한, 인기, 전체, 안목, 전문가, 유망, 투자자, 세분, 무엇, 특성]",2
998,"[사례, 실전, 사건, 대한, 등기, 우선, 내용증명, 발급, 특수, 소송]",1


In [20]:
lda_df2['cluster'] = kmeans.labels_
lda_df2

/var/folders/r9/tknpwgr56sj838cg0tp52mn00000gp/T/ipykernel_8385/3392653776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lda_df2['cluster'] = kmeans.labels_


,title,isbn13,itemid,book_introduce,cluster
0,역행자 확장판 - 돈·시간·운명으로부터 완전한 자유를 얻는 7단계 인생 공략집,9788901272580,317300995,95퍼센트의 인간은 타고난 유전자와 본성의 꼭두각시로 살아간다 이들은 평생 돈 시간...,2
1,차가운 자본주의,9791192372464,319273965,자본주의 사회를 살아가는 우리는 누구나 부자를 꿈꾼다 돈이 인생의 전부까지는 아니어...,2
2,사장학개론,9791188331888,313250109,한국과 미국 전 세계를 오가며 사장을 가르치는 사장 으로 알려진 돈의 속성 의 저자...,2
3,박태웅의 AI 강의 - 챗GPT의 실체부터 AI의 진화와 미래까지 인간의 뇌를 초월...,9791157846764,318305026,2022년 말 GPT 3 5를 기반으로 한 챗GPT가 출시되었다 그리고 모두가 알다...,2
4,벤 버냉키의 21세기 통화 정책 - 연방준비제도 - 대 인플레이션에서 코로나 팬데믹까지,9791192389219,318170803,오늘날 세계 경제는 긴밀하게 연결되어 있으며 그중에서도 미국 경제의 영향력은 너무나...,2
...,...,...,...,...,...
995,그림으로 이해하는 경제사상,9788957691885,34339005,경제학사에 등장하는 주요 경제학자들의 경제학 용어 를 알기 쉬운 그림과 함께 실은 ...,2
996,채권기초 - 5판,9791192515113,303043569,채권기초는 채권을 처음 접하는 독자들이 채권 전반에 대해 이해하고 또한 채권이론을 ...,2
997,"퍼펙트 입지 - 건물 투자, 입지를 알아야 돈이 보인다",9791197721427,314899995,당신은 왜 건물주가 되지 못할까 라는 화두를 던진 바 있는 박준연 대표의 건물 투자...,2
998,부동산 경매.공매 특수물건 투자 비법 - 실전 사례로 알려주는,9791164842438,271720328,법정지상권 내용과 지분 경매와 공유자 우선매수를 다루었으며 농지와 농지취득자격증명 ...,1


In [ ]:
# 클러스터별로 다시 LDA
# 클러스터 하나 전체 문서에 대해 토픽들 뽑기 10개 -> 클러스터에 대한 특징 알 수 있음 
# 지금까지
# 하나의 도서당 토픽(토픽별 상위 1번째 단어) 10개 단어 뽑아서
# 그 토픽들에 대해 군집분석 -> 책들이 어느 군집에 할당됨

In [21]:
cluster_df = lda_df2.copy()

In [22]:
cluster_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           1000 non-null   object
 1   isbn13          1000 non-null   object
 2   itemid          1000 non-null   int64 
 3   book_introduce  1000 non-null   object
 4   cluster         1000 non-null   int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 35.3+ KB


In [24]:
cluster_df['cluster'].value_counts()

cluster
2    826
0     89
1     85
Name: count, dtype: int64

In [62]:
clu_1 = cluster_df[cluster_df['cluster'] == 1]
clu_2 = cluster_df[cluster_df['cluster'] == 2]
clu_0 = cluster_df[cluster_df['cluster'] == 0]

In [63]:
clu_1 = clu_1.reset_index(drop=True)
clu_2 = clu_2.reset_index(drop=True)
clu_0 = clu_0.reset_index(drop=True)

In [35]:
clu_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           85 non-null     object
 1   isbn13          85 non-null     object
 2   itemid          85 non-null     int64 
 3   book_introduce  85 non-null     object
 4   cluster         85 non-null     int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 3.1+ KB


In [64]:
text_1 = clu_1['book_introduce']
text_2 = clu_2['book_introduce']
text_0 = clu_0['book_introduce']

In [37]:
text_1

0     재테크에 관한 진정성 있는 따뜻한 조언으로 Z세대의 마음을 사로잡은 유튜브 크리에이...
1     지난해 11월 출간되어 큰 사랑을 받았던 주택청약의 모든 것 이 출간 8개월 만에 ...
2     하버드케네디스쿨 비즈니스스쿨 등 하버드대학교에서 경제학 교수를 역임하며 당대 최고의...
3     세계적인 부자학의 대가인 토머스 스탠리 박사는 자수성가한 부자들은 어떻게 부를 축적...
4     절망에 대한 해독제이자 더 나은 미래를 향한 로드맵이다 전 세계의 과학자와 경제학자...
                            ...                        
80    1990년대부터 국내에서도 본격적으로 조직문화의 중요성이 부각됐다 사내문화 기업문화...
81    대한민국의 미래 동력이 될 부의 내비게이터를 지향하는 경제 전망서다 2022 미래지...
82    보험에 대한 그릇된 인식을 해소하고 사고와 질병 등으로 어려움에 처했을 때 구독자분...
83    정체된 조직을 혁신하기 위해 오늘날 추구해야 할 바람직한 리더십을 제시하는 경제경영...
84    법정지상권 내용과 지분 경매와 공유자 우선매수를 다루었으며 농지와 농지취득자격증명 ...
Name: book_introduce, Length: 85, dtype: object

In [47]:
# cluster 1 
from konlpy.tag import * 
from tqdm import tqdm


# 토큰화
okt = Okt()

tokens = []
for i in tqdm(range(len(text_1))):
    token = okt.nouns(text_1[i])
    tokens.append(token)

# 불용어 
stopwords = ['와','에','으로','도','를','과','는','의','가','이','은','이','있','하','것','들','그','되','수','이','보','않','없','나','사람','주','아니','등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나','받','못하','일','그런','또','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓']

drop_stopwords = []
for j in range(len(text_1)):
    stop = [word for word in tokens[j] if word not in stopwords]
    drop_stopwords.append(stop)

    
detokenized_doc = []
for i in range(len(text_1)):
    t = ' '.join(drop_stopwords[i])
    detokenized_doc.append(t)    

    
# 토픽 확인 함수
def get_topics(components, feature_names, n=1000):
    features_li = []
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1))
        print([(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        
        features = []
        for i in topic.argsort()[:-n - 1:-1]:
            feature = feature_names[i]
            features.append(feature)
            
        features_li.append(features)
    return features_li
        
    

# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words=stopwords, max_features=1000)
X = vectorizer.fit_transform(detokenized_doc)

lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)

terms = list(vectorizer.vocabulary_.keys())

topics = get_topics(lda_model.components_,terms)    

100%|███████████████████████████████████████████| 85/85 [00:03<00:00, 26.03it/s]


Topic 1:
[('코인', 1.24), ('가치', 1.2), ('트레이딩', 1.14), ('요소', 1.11), ('단어', 1.11), ('직무', 1.08), ('인내', 1.07), ('핀터레스트', 1.07), ('노하우', 1.07), ('시리즈', 1.07), ('인구', 1.06), ('평가', 1.06), ('느낌', 1.06), ('보고', 1.05), ('취직', 1.05), ('직장', 1.04), ('가슴', 1.04), ('한계', 1.04), ('지난', 1.03), ('인터넷', 1.03), ('플레이', 1.03), ('집대성', 1.03), ('셀러', 1.03), ('로드맵', 1.03), ('검증', 1.02), ('성취', 1.02), ('운영', 1.02), ('브랜딩', 1.02), ('공식', 1.02), ('순간', 1.01), ('찾기', 1.01), ('금융위기', 1.01), ('트릭', 1.01), ('해석', 1.01), ('가장', 1.01), ('체험', 1.01), ('매점', 1.01), ('출발', 1.01), ('장의', 1.01), ('소셜미디어', 1.0), ('획기', 1.0), ('농지', 1.0), ('단순', 1.0), ('사랑', 1.0), ('자본', 1.0), ('이제', 1.0), ('더욱', 1.0), ('미리', 1.0), ('여러분', 0.99), ('원인', 0.99), ('보유', 0.99), ('백만장자', 0.99), ('일반', 0.99), ('원칙', 0.99), ('신간', 0.98), ('정부', 0.98), ('권위', 0.98), ('창해', 0.98), ('서양', 0.98), ('별로', 0.98), ('용기', 0.98), ('당시', 0.98), ('상호작용', 0.97), ('전매', 0.97), ('채권', 0.97), ('자료', 0.97), ('편의', 0.97), ('수익률', 0.97), ('화술', 0.97), ('시중', 0.97

In [52]:
# cluster 2 
from konlpy.tag import * 
from tqdm import tqdm


# 토큰화
okt = Okt()

tokens = []
for i in tqdm(range(len(text_2))):
    token = okt.nouns(text_2[i])
    tokens.append(token)

# 불용어 
stopwords = ['와','에','으로','도','를','과','는','의','가','이','은','이','있','하','것','들','그','되','수','이','보','않','없','나','사람','주','아니','등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나','받','못하','일','그런','또','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓']

drop_stopwords = []
for j in range(len(text_2)):
    stop = [word for word in tokens[j] if word not in stopwords]
    drop_stopwords.append(stop)

    
detokenized_doc = []
for i in range(len(text_2)):
    t = ' '.join(drop_stopwords[i])
    detokenized_doc.append(t)    

    
# 토픽 확인 함수
def get_topics(components, feature_names, n=1000):
    features_li = []
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1))
        print([(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        
        features = []
        for i in topic.argsort()[:-n - 1:-1]:
            feature = feature_names[i]
            features.append(feature)
            
        features_li.append(features)
    return features_li
        
    

# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words=stopwords, max_features=1000)
X = vectorizer.fit_transform(detokenized_doc)

lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)

terms = list(vectorizer.vocabulary_.keys())

topics = get_topics(lda_model.components_,terms) 

100%|█████████████████████████████████████████| 826/826 [00:25<00:00, 31.85it/s]


Topic 1:
[('역대', 7.81), ('사상', 7.21), ('국내', 6.9), ('준비', 6.31), ('시점', 6.15), ('진실', 5.97), ('데이터', 5.85), ('대한', 5.66), ('함정', 5.35), ('실행', 5.29), ('정체', 4.95), ('경제', 4.9), ('모빌리티', 4.86), ('임대', 4.67), ('유통', 4.49), ('온라인', 4.45), ('널리', 4.33), ('추월', 4.31), ('메시지', 4.31), ('기준', 4.29), ('한계', 4.26), ('원리', 4.25), ('주요', 4.22), ('통찰', 4.22), ('마지막', 4.18), ('이나', 4.1), ('근거', 4.04), ('가치', 3.98), ('체계', 3.94), ('월급', 3.84), ('실전', 3.83), ('관점', 3.77), ('별로', 3.74), ('아마존', 3.74), ('불어', 3.68), ('리뷰', 3.67), ('높이', 3.66), ('여성', 3.65), ('시대', 3.6), ('직장인', 3.57), ('세제', 3.55), ('이야기', 3.55), ('공개', 3.43), ('바로', 3.41), ('개발', 3.37), ('상식', 3.36), ('투자가', 3.24), ('실습', 3.24), ('만화', 3.15), ('소비자', 3.14), ('메타', 3.13), ('공간', 3.09), ('부록', 3.07), ('입지', 3.07), ('최초', 3.07), ('시도', 3.07), ('결과', 2.99), ('비트코인', 2.97), ('공통점', 2.95), ('구체', 2.94), ('즉시', 2.91), ('스토리텔링', 2.9), ('지난', 2.88), ('고유', 2.87), ('영향', 2.84), ('대화', 2.84), ('목표', 2.82), ('하와이', 2.77), ('사태', 2.77), ('위해', 2.73

In [66]:
# cluster 0
from konlpy.tag import * 
from tqdm import tqdm


# 토큰화
okt = Okt()

tokens = []
for i in tqdm(range(len(text_0))):
    token = okt.nouns(text_0[i])
    tokens.append(token)
    
    
# 불용어 
stopwords = ['와','에','으로','도','를','과','는','의','가','이','은','이','있','하','것','들','그','되','수','이','보','않','없','나','사람','주','아니','등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나','받','못하','일','그런','또','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨','시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개','전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','놓']

drop_stopwords = []
for j in range(len(text_0)):
    stop = [word for word in tokens[j] if word not in stopwords]
    drop_stopwords.append(stop)
    
    
detokenized_doc = []
for i in range(len(text_0)):
    t = ' '.join(drop_stopwords[i])
    detokenized_doc.append(t)    

    
# 토픽 확인 함수
def get_topics(components, feature_names, n=1000):
    features_li = []
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1))
        print([(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        
        features = []
        for i in topic.argsort()[:-n - 1:-1]:
            feature = feature_names[i]
            features.append(feature)
            
        features_li.append(features)
    return features_li
        
    

# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words=stopwords, max_features=1000)
X = vectorizer.fit_transform(detokenized_doc)

lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)

terms = list(vectorizer.vocabulary_.keys())

topics = get_topics(lda_model.components_,terms) 

100%|███████████████████████████████████████████| 89/89 [00:03<00:00, 24.02it/s]

Topic 1:
[('관리자', 1.25), ('호응', 1.2), ('집필', 1.16), ('신조어', 1.16), ('실무', 1.15), ('종잣돈', 1.15), ('일군', 1.14), ('활약', 1.14), ('크기', 1.13), ('확보', 1.11), ('직원', 1.1), ('분야', 1.09), ('글로벌', 1.09), ('관계', 1.08), ('강의', 1.08), ('블로그', 1.07), ('엔지니어', 1.07), ('하루하루', 1.07), ('등장', 1.06), ('강사', 1.06), ('자동', 1.06), ('금융위기', 1.06), ('부터', 1.05), ('성향', 1.05), ('에어비앤비', 1.04), ('하루', 1.04), ('혼란', 1.04), ('세상', 1.03), ('기억', 1.03), ('작업', 1.03), ('조장', 1.03), ('크게', 1.03), ('정보', 1.03), ('프레임', 1.02), ('재능', 1.02), ('목표', 1.02), ('무용가', 1.02), ('협업', 1.02), ('이론', 1.02), ('플랫폼', 1.01), ('장담', 1.01), ('실체', 1.01), ('예제', 1.01), ('역설', 1.01), ('회장', 1.01), ('퇴사', 1.01), ('수단', 1.01), ('최근', 1.01), ('한마디', 1.0), ('험담', 1.0), ('라면', 1.0), ('불변', 1.0), ('부담', 1.0), ('독보', 1.0), ('학회', 1.0), ('회사', 1.0), ('공매', 1.0), ('경험', 1.0), ('실적', 0.99), ('당신', 0.99), ('경제활동', 0.99), ('문장', 0.99), ('디자인', 0.99), ('성과', 0.99), ('구글', 0.99), ('신경', 0.99), ('카피', 0.98), ('유효', 0.98), ('최대한', 0.98), ('확장', 0.98), 

In [ ]:
# 군집만이랑
# 토픽당 상위 1개 뽑지 않은것